In [16]:
# imports
import myvariant
import pandas as pd
import numpy as np
mv = myvariant.MyVariantInfo()

# Import DNA

In [17]:
feather_file_path = "data/sm_feather";
df = pd.read_feather(feather_file_path, use_threads=True);

## Narrow it down
#df = df[df.chromosome == "6"]

# GeneLoc map region around 10MB CYP21 : 27038265 --> 37041670
#df = df[(df['position'] >= 27038265) & (df['position'] <= 37041670)] # 7000 entries
#df = df[(df['position'] >= 33038265) & (df['position'] <= 35041670)] # 600 entries

# Remove unneeeded columns
del df['position']
del df['chromosome']

# df_Full.to_feather(feather_file_path);
df.reset_index(drop=True)

# Remove i-rsids
df.drop(df[df.rsid.str.contains('i')].index, inplace=True)

# Move to a list for mv.querymany 
your_rsid_list = df['rsid'].values.tolist()

df

,rsid,genotype
0,rs548049170,TT
1,rs13328684,--
2,rs9283150,AA
4,rs116587930,GG
5,rs3131972,GG
...,...,...
638342,rs41479950,A
638355,rs2853817,T
638363,rs35134837,T
638373,rs2853513,C


In [ ]:
res_original = mv.querymany(list(set(your_rsid_list)), scopes=['dbsnp.rsid', 'clinvar.rsid', 'wellderly.adviser_score'], fields='wellderly', as_dataframe=True)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

querying 271001-272000...done.
querying 272001-273000...done.
querying 273001-274000...done.
querying 274001-275000...done.
querying 275001-276000...done.
querying 276001-277000...done.
querying 277001-278000...done.
querying 278001-279000...done.
querying 279001-280000...done.
querying 280001-281000...done.
querying 281001-282000...done.
querying 282001-283000...done.
querying 283001-284000...done.
querying 284001-285000...done.
querying 285001-286000...done.
querying 286001-287000...done.
querying 287001-288000...done.
querying 288001-289000...done.
querying 289001-290000...done.
querying 290001-291000...done.
querying 291001-292000...done.
querying 292001-293000...done.
querying 293001-294000...done.
querying 294001-295000...done.
querying 295001-296000...done.
querying 296001-297000...done.
querying 297001-298000...done.
querying 298001-299000...done.
querying 299001-300000...done.
querying 300001-301000...done.
querying 301001-302000...done.
querying 302001-303000...done.
querying

In [1]:
res = res_original 
res.to_csv('data/23andme_v5_FULL.csv', encoding='utf-8', index=False)

res.drop(['wellderly.original_aa', 'wellderly.polyphen', 'wellderly.protein_pos', 
          'wellderly.sift', 'wellderly.clinvar', 'wellderly.genotypes', 'wellderly.chrom', 
          'wellderly.hg19.end', '_score', 'wellderly._license','wellderly.hg19.start', 
          'wellderly.pos', 'wellderly.vartype', 'notfound', 'wellderly.allele_aa', 
          'wellderly.coding_impact', 'wellderly.splice_site_pred', 'wellderly.ref',
          'level_0', 'index'
         ], inplace=True, axis=1, errors='ignore')

# Drop ALL non-significant snps
#res = res.dropna(subset=['wellderly.adviser_score'])

res

NameError: name 'res_original' is not defined

In [5]:
# Flatten nested dictionary list object
df_p = res['wellderly.alleles'].apply(pd.Series)
df_p_0 = df_p[0].apply(pd.Series)
df_p_1 = df_p[1].apply(pd.Series)

df_p_0.columns = ['0','Allele', 'Freq']
df_p_1.columns = ['0','Allele', 'Freq']

# Concat them back together
res = pd.concat([res, df_p_0, df_p_1], axis=1)
res.drop(['0','wellderly.alleles'], inplace=True, axis=1, errors='ignore')


In [6]:
# Drop ID's which aren't useful
res = res.dropna(subset=['_id'])
res.drop(res[res._id.str.contains(']$')].index, inplace=True)
#del res['index']

res

,_id,wellderly.adviser_score,wellderly.alt,wellderly.gene,Allele,Freq,Allele,Freq
query,,,,,,,,
rs72987544,chr6:g.137236687G>A,"5~PEX7~Uncommon, Predicted Neutral",A,PEX7,A,0.0175,G,0.9825
rs117128327,chr6:g.33241546C>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rs2092556,chr6:g.135899344C>T,NaN,T,C6orf217,C,0.0825,T,0.9175
rs191370704,chr6:g.95918878C>T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rs191370704,chr6:g.95918878C>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
rs77325751,chr6:g.132879437A>C,NaN,C,TAAR8,A,0.9750,C,0.0250
rs11242905,chr6:g.474405C>T,NaN,T,EXOC2,C,0.1800,T,0.8200
rs948522,chr6:g.109169044C>T,NaN,T,ARMC2,C,0.8225,T,0.1775


# Reorder Columns

In [7]:
res

,_id,wellderly.adviser_score,wellderly.alt,wellderly.gene,Allele,Freq,Allele,Freq
query,,,,,,,,
rs72987544,chr6:g.137236687G>A,"5~PEX7~Uncommon, Predicted Neutral",A,PEX7,A,0.0175,G,0.9825
rs117128327,chr6:g.33241546C>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rs2092556,chr6:g.135899344C>T,NaN,T,C6orf217,C,0.0825,T,0.9175
rs191370704,chr6:g.95918878C>T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rs191370704,chr6:g.95918878C>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
rs77325751,chr6:g.132879437A>C,NaN,C,TAAR8,A,0.9750,C,0.0250
rs11242905,chr6:g.474405C>T,NaN,T,EXOC2,C,0.1800,T,0.8200
rs948522,chr6:g.109169044C>T,NaN,T,ARMC2,C,0.8225,T,0.1775


In [ ]:
# Move rsid/genotype to front

# Reset index, push 'query'
res.reset_index(level=0, inplace=True)
res['rsid'] = res['query']
ohres.drop(['query'], inplace=True, axis=1, errors='ignore')


cols = res.columns.tolist()
print(cols)
cols.insert(0, cols.pop(cols.index('rsid')))
cols.insert(1, cols.pop(cols.index('wellderly.gene')))
cols.insert(2, cols.pop(cols.index('wellderly.alt')))
#cols.insert(3, cols.pop(cols.index('wellderly.ref')))
cols.insert(4, cols.pop(cols.index('wellderly.adviser_score')))
res = res.reindex(columns= cols)





In [9]:
res.columns =  ['rsid', 'gene', 'alt', 'adviser_score', '_id', 'Allele', 'Freq', 'Allele', 'Freq']


['_id', 'wellderly.adviser_score', 'wellderly.alt', 'wellderly.gene', 'Allele', 'Freq', 'Allele', 'Freq']


In [10]:
res = res.fillna("")

res

,rsid,gene,alt,adviser_score,_id,Allele1,Freq1,Allele2,Freq2
0,chr6:g.137236687G>A,"5~PEX7~Uncommon, Predicted Neutral",A,PEX7,A,0.0175,G,0.9825,rs72987544
1,chr6:g.33241546C>A,,,,,,,,rs117128327
2,chr6:g.135899344C>T,,T,C6orf217,C,0.0825,T,0.9175,rs2092556
3,chr6:g.95918878C>T,,,,,,,,rs191370704
4,chr6:g.95918878C>A,,,,,,,,rs191370704
...,...,...,...,...,...,...,...,...,...
50756,chr6:g.132879437A>C,,C,TAAR8,A,0.975,C,0.025,rs77325751
50757,chr6:g.474405C>T,,T,EXOC2,C,0.18,T,0.82,rs11242905
50758,chr6:g.109169044C>T,,T,ARMC2,C,0.8225,T,0.1775,rs948522
50759,chr6:g.90021186T>C,"5~GABRR2~Uncommon, Predicted Neutral",C,GABRR2,C,0.0375,T,0.9625,rs79482608


In [15]:
res.to_csv('data/23andme_v5_ch6.csv', encoding='utf-8', index=False)

# Import from CSV

In [13]:
#res = pd.read_csv("data/10mb_cyp21.csv")
#res = pd.read_csv("data/23andme_FULL_SCRAPE.csv") 
res = pd.read_csv("data/23andme_v5_ch6.csv")

res

,rsid,gene,alt,adviser_score,_id,Allele1,Freq1,Allele2,Freq2
0,chr6:g.137236687G>A,"5~PEX7~Uncommon, Predicted Neutral",A,PEX7,A,0.0175,G,0.9825,rs72987544
1,chr6:g.33241546C>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs117128327
2,chr6:g.135899344C>T,NaN,T,C6orf217,C,0.0825,T,0.9175,rs2092556
3,chr6:g.95918878C>T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs191370704
4,chr6:g.95918878C>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs191370704
...,...,...,...,...,...,...,...,...,...
50756,chr6:g.132879437A>C,NaN,C,TAAR8,A,0.9750,C,0.0250,rs77325751
50757,chr6:g.474405C>T,NaN,T,EXOC2,C,0.1800,T,0.8200,rs11242905
50758,chr6:g.109169044C>T,NaN,T,ARMC2,C,0.8225,T,0.1775,rs948522
50759,chr6:g.90021186T>C,"5~GABRR2~Uncommon, Predicted Neutral",C,GABRR2,C,0.0375,T,0.9625,rs79482608


In [14]:
# Drop ID's which aren't useful
res = res.dropna(subset=['_id'])
res.drop(res[res._id.str.contains(']$')].index, inplace=True)
#del res['index']

res

# Reset index, push 'query'
#res.reset_index(level=0, inplace=True)
#res['rsid'] = res['query']
#del res['query']

res

/usr/local/lib64/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,rsid,gene,alt,adviser_score,_id,Allele1,Freq1,Allele2,Freq2
0,chr6:g.137236687G>A,"5~PEX7~Uncommon, Predicted Neutral",A,PEX7,A,0.0175,G,0.9825,rs72987544
2,chr6:g.135899344C>T,NaN,T,C6orf217,C,0.0825,T,0.9175,rs2092556
5,chr6:g.149749403G>A,NaN,A,MAP3K7IP2,A,0.4175,G,0.5825,rs392917
6,chr6:g.28485680A>C,NaN,C,GPX6,A,0.9950,C,0.0050,rs423118
9,chr6:g.98980147A>G,NaN,G,POU3F2,A,0.1850,G,0.8150,rs211222
...,...,...,...,...,...,...,...,...,...
50756,chr6:g.132879437A>C,NaN,C,TAAR8,A,0.9750,C,0.0250,rs77325751
50757,chr6:g.474405C>T,NaN,T,EXOC2,C,0.1800,T,0.8200,rs11242905
50758,chr6:g.109169044C>T,NaN,T,ARMC2,C,0.8225,T,0.1775,rs948522
50759,chr6:g.90021186T>C,"5~GABRR2~Uncommon, Predicted Neutral",C,GABRR2,C,0.0375,T,0.9625,rs79482608


In [ ]:
res